In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import joblib

# Load data
df = pd.read_csv('house_data.csv')

# Drop irrelevant columns (adjust based on your data)
df = df.drop(columns=['SizeRank', 'RegionType', 'StateName'], errors='ignore')

# If date columns exist, melt into long format
date_columns = [col for col in df.columns if any(str(year) in col for year in range(2000, 2030))]
if date_columns:
    id_vars = [col for col in df.columns if col not in date_columns]
    df = pd.melt(df, id_vars=id_vars, value_vars=date_columns, var_name='Date', value_name='Price')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

# Define features and target
target = 'Price' if 'Price' in df.columns else df.columns[-1]
X = df.drop(columns=[target])
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
])

# Train model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")
print(f"RMSE: ${np.sqrt(mean_squared_error(y_test, y_pred)):,.2f}")

# Save model
joblib.dump(model, 'house_price_model1.pkl')
print("Model saved as 'house_price_model1.pkl'")

R² Score: -0.05
RMSE: $1.51
Model saved as 'house_price_model1.pkl'
